# globals



In [1]:
n_classes = 10
actions = ['sister','hurry','hungry','meal','brother','tree','heavy','cry','family','wise']
colors = [
    (245,117,16),
    (117,245,16),
    (16,117,245)
]

def softmax(x):    
    f_x = np.exp(x) / np.sum(np.exp(x))
    return f_x

def arg_max(array):
    arg_max = np.argmax(array)
    return arg_max,array[arg_max]

# pytorch Model

In [2]:
import pandas as pd
import os
import cv2
import numpy as np
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from PIL import Image


def convert_relu_to_swish(model: nn.Module):
    for child_name, child in model.named_children():
        if isinstance(child, nn.ReLU):
            setattr(model, child_name, nn.SiLU(True))
        else:
            convert_relu_to_swish(child)


            
            
class Swish(nn.Module):
    def __init(self):
        super().__init__()

    def forward(self, x):
        return x.mult_(torch.sigmoid(x))
    
    
    
class r2plus1d_18(nn.Module):
    def __init__(self, pretrained=True, n_classes=3, dropout_p=0.5):
        super(r2plus1d_18, self).__init__()
        self.pretrained = pretrained
        self.n_classes = n_classes

        model = torchvision.models.video.r2plus1d_18(pretrained=self.pretrained)
        modules = list(model.children())[:-1]
        self.r2plus1d_18 = nn.Sequential(*modules)
        convert_relu_to_swish(self.r2plus1d_18)
        self.fc1 = nn.Linear(model.fc.in_features, self.n_classes)
        self.dropout = nn.Dropout(dropout_p, inplace=True)

    def forward(self, x):
        # (b, f, c, h, w) = x.size()
        # x = x.view(b, c, f, h, w)

        out = self.r2plus1d_18(x)
        out = out.flatten(1)
        out = self.dropout(out)
        out = self.fc1(out)

        return out
    
    
h, w = 128, 128
mean = [0.43216, 0.394666, 0.37645]
std = [0.22803, 0.22145, 0.216989]



pytorch_model = r2plus1d_18(pretrained=False, n_classes=n_classes)
best_checkpoint = torch.load("pytorch_weights.tar")
pytorch_model.load_state_dict(best_checkpoint["model_state_dict"])


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pytorch_model = pytorch_model.to(device)




In [3]:
resize_transform   = transforms.Resize((h, w))
totensor_transform  = transforms.ToTensor()
normalize_transform = transforms.Normalize(mean, std)

class PytorchPredictor:
    def __init__(self,model,device):
        self.model = model
        self.device = device
        self.sequence = []
        
    
    def can_predict(self):
        return len(self.sequence) == 16
    
    def add_frame(self,frame):
        
        new_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        new_frame = Image.fromarray(new_frame)
        new_frame = resize_transform(new_frame)
        new_frame = totensor_transform(new_frame)
        new_frame = normalize_transform(new_frame).to(self.device)
        
        self.sequence.append(new_frame)
        self.sequence = self.sequence[-16:]
        
        
    def predict(self):
        seq = torch.stack(self.sequence).to(self.device)
        seq = torch.unsqueeze(seq, dim=0).permute(0, 2, 1, 3, 4)
        
        with torch.no_grad():
            self.model.eval()
            res = self.model(seq)
            res = res.cpu().detach().numpy()[0]
            return softmax(res)
        
        
        

# Keras model

In [6]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
import tensorflow as tf
tf.config.set_visible_devices([], 'GPU')

In [8]:
tf.test.gpu_device_name()

2022-01-13 15:11:50.486843: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-13 15:11:50.487227: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-13 15:11:50.487419: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-13 15:11:50.487654: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-13 15:11:50.487845: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from S

'/device:GPU:0'

In [9]:
import tensorflow as tf

tf.config.set_visible_devices([], 'GPU')

if tf.test.gpu_device_name():
    print('GPU found')
else:
    print("No GPU found")

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

import numpy as np
import os
import cv2 
import mediapipe as mp
import tensorflow as tf 
from tensorflow import keras
import matplotlib.pyplot as plt
import time
import pandas as pd

GPU found
Num GPUs Available:  1


2022-01-13 15:11:57.932349: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-13 15:11:57.932778: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-13 15:11:57.933087: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-13 15:11:57.933458: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-13 15:11:57.933785: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from S

In [10]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

holistic = mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5)


num_hand_marks = 21
num_pose_marks = 33


pose_selected_landmarks = [
    [0,2,5,11,13,15,12,14,16],
    [0,2,4,5,8,9,12,13,16,17,20],
    [0,2,4,5,8,9,12,13,16,17,20],
]

def draw_updated_styled(image,results):
    image_rows, image_cols, _ = image.shape
    
    original_landmarks = [
        results.pose_landmarks,
        results.left_hand_landmarks,
        results.right_hand_landmarks
    ]

    
    for shape in range(3):
        if(original_landmarks[shape]):
            lis = original_landmarks[shape].landmark
            for idx in pose_selected_landmarks[shape]:
                point = lis[idx]
                landmark_px = mp_drawing._normalized_to_pixel_coordinates(point.x, point.y,
                                                           image_cols, image_rows)

                cv2.circle(image, landmark_px, 2, (0,0,255),
                         4)     
                
def extract_keypoints(results):
    
    original_landmarks = [
        results.pose_landmarks,
        results.left_hand_landmarks,
        results.right_hand_landmarks
    ]
    
    outputs = []
    for shape in range(3):
        if(original_landmarks[shape]):
            lis = original_landmarks[shape].landmark
            pose = np.array([ [lis[res].x,lis[res].y] for res in pose_selected_landmarks[shape] ]).flatten()
        else:
            pose = np.zeros(len(pose_selected_landmarks[shape])*2)
        outputs.append(pose)
    return np.concatenate([outputs[0],outputs[1],outputs[2]])



# holistic model process image and return the results as keypoints
def mediapipe_detection(image,model):
    image  = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image  = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
    return image,results
    
    


def draw_landmark_from_array(image,keyPoints):
    image_rows, image_cols, _ = image.shape
    
    
    for i in range(len(keyPoints)//2):
        x = keyPoints[i*2]
        y = keyPoints[i*2+1]
        if(x!=0 and y!=0): 
            landmark_px = mp_drawing._normalized_to_pixel_coordinates(x,y,
                                                       image_cols, image_rows)
            cv2.circle(image, landmark_px, 2, (0,0,255),
                     4)

                


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense,Input,Dropout
from tensorflow.keras.models import Model



def get_model():
    input_layer = Input(shape=(16,62))
    layer = LSTM(64,return_sequences=True,activation="relu")(input_layer)
    layer = LSTM(128,return_sequences=True,activation="relu")(layer)
    layer = LSTM(96,return_sequences=False,activation="relu")(layer)
    layer = Dense(64,activation="relu")(layer)
    layer = Dense(len(actions),activation="softmax")(layer)

    
    model = Model(inputs=input_layer,outputs=layer)
    model.compile(optimizer="Adam", loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model



keras_weights_dir = os.path.join("keras_weights")
best_model_file_name = os.path.join(keras_weights_dir,"V1.h5")
keras_model = get_model()
keras_model.load_weights(os.path.join(best_model_file_name))

In [12]:
class KerasPredictor:
    def __init__(self,model):
        self.model = model
        self.sequence = []
        
    
    def can_predict(self):
        return len(self.sequence) == 16
    
    def add_frame(self,frame):
        
        f2 = cv2.resize(frame,(512,512))
        image, results = mediapipe_detection(f2, holistic)
        keypoints = extract_keypoints(results)
        
        self.sequence.append(keypoints)
        self.sequence = self.sequence[-16:]
        
        
        
    def predict(self):
        return self.model.predict(np.expand_dims(self.sequence, axis=0))[0]
        
        
        

# Real time

In [13]:

def prob_viz(res, actions, input_frame, colors):
    l = len(colors)
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        prob = max(0,prob)
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100)*5, 90+num*40), colors[num%l], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame


In [14]:

pytorch_predictor = PytorchPredictor(model=pytorch_model,device=device)
keras_predictor = KerasPredictor(model=keras_model)



sentence = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(0)
# Set mediapipe model 





while cap.isOpened():

    # Read feed
    ret, frame = cap.read()
    if(not ret):
        break

    pytorch_predictor.add_frame(frame)
    keras_predictor.add_frame(frame)
    

    arg_max=-1
    #if pytorch_predictor.can_predict() and keras_predictor.can_predict():
    if pytorch_predictor.can_predict() :   
        res1 = pytorch_predictor.predict()
        res2 = keras_predictor.predict()
        res = res1 + res2
        # res = res1 
            
        arg_max = np.argmax(res)
            
            
        predictions.append(arg_max)


    #3. Viz logic
        if np.unique(predictions[-2:])[0]==arg_max: 
            if res[arg_max] > threshold: 

                if len(sentence) > 0: 
                    if actions[arg_max] != sentence[-1]:
                        sentence.append(actions[arg_max])
                else:
                    sentence.append(actions[arg_max])

        if len(sentence) > 5: 
            sentence = sentence[-5:]

        # Viz probabilities
        frame = prob_viz(res, actions, frame, colors)

    cv2.rectangle(frame, (0,0), (640, 40), (245, 117, 16), -1)
    cv2.putText(frame, ' '.join(sentence), (3,30), 
                   cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

    # Show to screen
    
    cv2.imshow('OpenCV Feed', frame)

    # Break gracefully
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

In [10]:
cap.release()
cv2.destroyAllWindows()

In [ ]:
# the next line to clear nvidia ca
sudo rm -rf ~/.nv/